# Creating a Dutch Cities dataset

The wikipedia page [List of largest cities](https://en.wikipedia.org/wiki/List_of_largest_cities) has a list of the largest cities in the world.  
We are interested in table that contains the actual cities, and want the cityname, the nation, and its city proper population. 

In [Section 2](#2.-Scrape-list-of-largest-cities-from-Wikipedia), we use urllib to fetch the page, then BeautifulSoup to parse it and find the first (and only) sortable table.  
We also scrape the URL referencing the individual cities' pages.

In [Section 3](#3.-Adding-geopositioning-data) we scrape the wikipedia page for every individual city in order to get the cities geographic coordinates.  
In theory we could use this step to grab more information, like population density. However, since we want to demonstrate the use of the Foursquare API, we will not use such data.

In [Section 4](#4.-Getting-more-information-from-Foursquare) we use the Foursquare API to learn more about these cities. 
Specifically we ask for top recommendations in those cities, to get an idea of what is popular.

In [Section 5](#5.-Saving-the-dataframes) we export the created dataframe so we can import it in other notebooks.

## 1. Imports and such things

We use the following libraries to fetch and parse html pages, and to interact with the Foursquare API.

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import requests
import requests_cache

# Request_cache caches all calls to wikipedia and foursquare so we only have to do it once.
requests_cache.install_cache('requests_cache')

We use regular expressions, e.g. to extract population count and city coordinates from scraped webpages

In [2]:
import re

We use Pandas for building the dataframe.

In [3]:
import pandas as pd

## 2. Scrape list of Dutch cities from Wikipedia

The following code fetches a Wikipedia page.

In [6]:
# location of the wikipedia article
url = "https://nl.wikipedia.org/wiki/Tabel_van_Nederlandse_gemeenten"

# fetch the article
response = requests.get(url)
article = response.text

We use BeautifulSoup to parse the obtained HTML, and find the first sortable table. This is the table of Largest cities that dominates the Wikipedia article.

In [7]:
# parse with BeautifulSoup and find the first sortable table
soup = BeautifulSoup(article, 'html.parser')
table = soup.find('table', class_='sortable')

We create a new Pandas dataframe to hold our cities. We will also save its nation, its population count, and its URL.

In [8]:
# create an empty DataFrame
cols=["City", "Province", "Population", "Area", "Density", "Income", "URL", "FlagURL"]
df_cities = pd.DataFrame(columns=cols)
df_cities['Population'].astype(int)
df_cities['Area'].astype(float)
df_cities['Density'].astype(float)
df_cities['Income'].astype(int)

Series([], Name: Income, dtype: int64)

We can now traverse through the entire table and append the required data to our dataframe

In [9]:
# iterate trough all the rows in the table:
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue                            # skips first row with headings

    citycell = tds[0].find_all('a')
    if (len(citycell)==2):
        flagurl = "https://nl.wikipedia.org" + citycell[0]['href']
        url = "https://nl.wikipedia.org" + citycell[1]['href']
        city = citycell[1]['title']
    else:
        url = "https://nl.wikipedia.org" + citycell[0]['href']
        city = citycell[0]['title']
    
    try:
        province = tds[1].find_all('a')[1]['title'].replace('(hoofdbetekenis)','').replace('(provincie)','').replace('(Nederland)','').strip()
    except:
        province = "Geen"
        
    pop = int(re.compile(r'\[.*\]').sub("",tds[2].text).replace('(geschat)','').strip().replace('.','').replace('.',''))
    area = float(tds[3].text.strip().replace('.','').replace(',','.'))
    density = float(tds[4].text.strip().replace('.','').replace(',','.'))    
    try:    
        income = int(tds[5].text.strip().replace('.',''))   
    except ValueError:
        income = None
        
    df_cities = df_cities.append({
        'City': str(city), 
        'Province': str(province), 
        'Population': pop,
        'Area': area,
        'Density': density,
        'Income': income,
        'URL': str(url),
        'FlagURL': str(flagurl)
    }, ignore_index=True)

In [10]:
df_cities.drop_duplicates(keep='first', inplace=True)

In [11]:
print ("Shape: {}".format(df_cities.shape))
print ("There are {} unique city names".format(df_cities['City'].nunique()))
df_cities.head()

Shape: (383, 8)
There are 383 unique city names


,City,Province,Population,Area,Density,Income,URL,FlagURL
0,Aa en Hunze,Drenthe,25319,276.35,91.2,13400,https://nl.wikipedia.org/wiki/Aa_en_Hunze,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...
1,Aalburg,Noord-Brabant,13191,50.40,257.5,12500,https://nl.wikipedia.org/wiki/Aalburg,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...
2,Aalsmeer,Noord-Holland,31584,20.58,1512.7,14700,https://nl.wikipedia.org/wiki/Aalsmeer,https://nl.wikipedia.org/wiki/Bestand:Aalsmeer...
3,Aalten (gemeente),Gelderland,26943,96.57,278.7,11900,https://nl.wikipedia.org/wiki/Aalten_(gemeente),https://nl.wikipedia.org/wiki/Bestand:Flag_of_...
4,Achtkarspelen,Friesland,27897,102.61,272.7,11400,https://nl.wikipedia.org/wiki/Achtkarspelen,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...


In [12]:
# Scrape an individual cities page for its coordinates
def scrape_city_coords(url):
    print(url)
    article = requests.get(url).text
    reg = re.search(r'"lat":(.*?),"lon":(.*?)}', article)
    if reg is None:
        return 0,0
    lat = float(reg.group(1))
    lon = float(reg.group(2))
    return lat,lon

Next we will run this function on the URLs to create two new columns

In [13]:
df_cities["Latitude"], df_cities["Longitude"] = zip(*df_cities["URL"].map(scrape_city_coords))

https://nl.wikipedia.org/wiki/Aa_en_Hunze
https://nl.wikipedia.org/wiki/Aalburg
https://nl.wikipedia.org/wiki/Aalsmeer
https://nl.wikipedia.org/wiki/Aalten_(gemeente)
https://nl.wikipedia.org/wiki/Achtkarspelen
https://nl.wikipedia.org/wiki/Alblasserdam
https://nl.wikipedia.org/wiki/Albrandswaard
https://nl.wikipedia.org/wiki/Alkmaar_(gemeente)
https://nl.wikipedia.org/wiki/Almelo
https://nl.wikipedia.org/wiki/Almere
https://nl.wikipedia.org/wiki/Alphen_aan_den_Rijn_(gemeente)
https://nl.wikipedia.org/wiki/Alphen-Chaam
https://nl.wikipedia.org/wiki/Ameland_(hoofdbetekenis)
https://nl.wikipedia.org/wiki/Amersfoort_(hoofdbetekenis)
https://nl.wikipedia.org/wiki/Amstelveen
https://nl.wikipedia.org/wiki/Amsterdam_(gemeente)
https://nl.wikipedia.org/wiki/Apeldoorn_(gemeente)
https://nl.wikipedia.org/wiki/Appingedam
https://nl.wikipedia.org/wiki/Arnhem
https://nl.wikipedia.org/wiki/Assen
https://nl.wikipedia.org/wiki/Asten_(gemeente)
https://nl.wikipedia.org/wiki/Baarle-Nassau
https://nl.wik

In [14]:
df_cities.shape

(383, 10)

remove anything without coordinates

In [15]:
df_cities = df_cities[df_cities['Latitude'] != 0]
df_cities = df_cities[df_cities['Longitude'] != 0]
df_cities.dropna(axis=0, subset=['Latitude', 'Longitude'])
df_cities.shape

(377, 10)

In [16]:
df_cities.head()

,City,Province,Population,Area,Density,Income,URL,FlagURL,Latitude,Longitude
0,Aa en Hunze,Drenthe,25319,276.35,91.2,13400,https://nl.wikipedia.org/wiki/Aa_en_Hunze,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,53.000000,6.750000
1,Aalburg,Noord-Brabant,13191,50.40,257.5,12500,https://nl.wikipedia.org/wiki/Aalburg,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,51.752500,5.131944
2,Aalsmeer,Noord-Holland,31584,20.58,1512.7,14700,https://nl.wikipedia.org/wiki/Aalsmeer,https://nl.wikipedia.org/wiki/Bestand:Aalsmeer...,52.263889,4.762500
3,Aalten (gemeente),Gelderland,26943,96.57,278.7,11900,https://nl.wikipedia.org/wiki/Aalten_(gemeente),https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,51.925000,6.580833
4,Achtkarspelen,Friesland,27897,102.61,272.7,11400,https://nl.wikipedia.org/wiki/Achtkarspelen,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,53.216667,6.133333


## 4. Getting more information from Foursquare

This information is needed to connect with Foursquare API

In [17]:
CLIENT_ID = 'YPBVFDUZOP1M24BKCWGXIYZ3RFACOE3V35WSFY4DSCMRU44L' # your Foursquare ID
CLIENT_SECRET = 'VYHYTBSRIZBPYAOCP5ZEFV3YM4C40YEQCQWCUO4NC1JTPNJM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

We will ask Foursquare for the top picks in every city, then store its name, location and main category.

In [18]:
def getRecommendedVenues(cities, latitudes, longitudes):
    
    venues_list=[]
    for city, lat, lon in zip(cities, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&section=topPicks&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon,
            100)
            
        # make the GET request
        response = requests.get(url)
        if response.status_code == requests.codes.ok:
            results = response.json()["response"]['groups'][0]['items']
        else:
            print ("status was:" + str(response.status_code))
            print ("could not scrape " + url)
            break
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            city,
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                  'Venue',
                  'Venue ID',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, actually run the above function for every city.

In [19]:
df_venues = getRecommendedVenues(df_cities['City'], df_cities['Latitude'], df_cities['Longitude'])
print("all done")

all done


In [20]:
print ("Found {} venues for {} different cities".format(df_venues.shape[0], len(df_venues.groupby('City'))))
df_venues.head()

Found 16339 venues for 377 different cities


,City,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Aa en Hunze,Hof van Saksen,4be09a4f4c55b651217beab7,52.966395,6.674498,Resort
1,Aa en Hunze,Robin Hood Ribhouse,4b85585df964a5200b5831e3,52.957088,6.788140,Restaurant
2,Aa en Hunze,Zwembad | Hof van Saksen,4c3b2cb3d94dc9b63156ffe7,52.965237,6.676055,Pool
3,Aa en Hunze,Restaurant Hofsteenge,4bebf67ba9900f47505b1840,52.935398,6.672966,Café
4,Aa en Hunze,Hunebedcentrum,4c2f2541ac0ab71368b41b1e,52.931130,6.797330,History Museum


That's looking pretty awesome. Now let's prevent more scraping by saving the dataframe to a file.

## 5. Joining the dataframe

In [21]:
df_joined = pd.merge(df_venues, df_cities, on='City')
print ("The joined dataframe contains {} different cities".format(len(df_joined.groupby('City'))))
df_joined.head()

The joined dataframe contains 377 different cities


,City,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Province,Population,Area,Density,Income,URL,FlagURL,Latitude,Longitude
0,Aa en Hunze,Hof van Saksen,4be09a4f4c55b651217beab7,52.966395,6.674498,Resort,Drenthe,25319,276.35,91.2,13400,https://nl.wikipedia.org/wiki/Aa_en_Hunze,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,53.0,6.75
1,Aa en Hunze,Robin Hood Ribhouse,4b85585df964a5200b5831e3,52.957088,6.788140,Restaurant,Drenthe,25319,276.35,91.2,13400,https://nl.wikipedia.org/wiki/Aa_en_Hunze,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,53.0,6.75
2,Aa en Hunze,Zwembad | Hof van Saksen,4c3b2cb3d94dc9b63156ffe7,52.965237,6.676055,Pool,Drenthe,25319,276.35,91.2,13400,https://nl.wikipedia.org/wiki/Aa_en_Hunze,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,53.0,6.75
3,Aa en Hunze,Restaurant Hofsteenge,4bebf67ba9900f47505b1840,52.935398,6.672966,Café,Drenthe,25319,276.35,91.2,13400,https://nl.wikipedia.org/wiki/Aa_en_Hunze,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,53.0,6.75
4,Aa en Hunze,Hunebedcentrum,4c2f2541ac0ab71368b41b1e,52.931130,6.797330,History Museum,Drenthe,25319,276.35,91.2,13400,https://nl.wikipedia.org/wiki/Aa_en_Hunze,https://nl.wikipedia.org/wiki/Bestand:Flag_of_...,53.0,6.75


## 6. Saving the dataframes

We export the dataframes so we don't have to scrape again.  
First as CSV, as this is a very generic format.  
Secondly as pickle, since this is a quick way to import the dataframe again.

In [22]:
# export both dataframes as CSV
df_cities.to_csv('cities.csv')
df_venues.to_csv('venues.csv')
df_joined.to_csv('joined.cvs')

In [23]:
# export both dataframe as pickles
df_cities.to_pickle('cities.pickle')
df_venues.to_pickle('venues.pickle')
df_joined.to_pickle('joined.pickle')